In [16]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

In [17]:
videos = pd.read_csv("VideoList_Veracity.csv")
videos

,VideoID,Veracity
0,BF001_1PT,1
1,BF001_2NL,0
2,BF001_3NT,1
3,BF001_4PL,0
4,BF002_1PT,1
...,...,...
315,WM026_4PL,0
316,WM027_1PT,1
317,WM027_2NL,0
318,WM027_3NT,1


In [91]:
videos.dtypes

VideoID     object
Veracity     int64
dtype: object

In [4]:
from sklearn.model_selection import train_test_split

X = videos['VideoID']
y = videos['Veracity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Train set shape: (256,)
Test set shape: (64,)


In [19]:
import pandas as pd
import os
import shutil

# Read the CSV file containing video names and veracity labels
csv_file = '/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/train_videos.csv'  # Adjust the file name and path as needed
df = pd.read_csv(csv_file)

df['VideoID'] = df['VideoID'].apply(lambda x: x + '.wmv')

# Save the DataFrame back to CSV, overwriting the original file
df.to_csv(csv_file, index=False)

# Separate video names based on their veracity labels (truth and lie)
truth_videos = df[df['Veracity'] == 1]['VideoID'].tolist()
lie_videos = df[df['Veracity'] == 0]['VideoID'].tolist()




In [21]:
from sklearn.model_selection import train_test_split

# Split truth and lie videos into train and val sets, maintaining symmetry of labels
truth_train, truth_val = train_test_split(truth_videos, test_size=0.25, random_state=42)
lie_train, lie_val = train_test_split(lie_videos, test_size=0.25, random_state=42)

# Create new folders for train and val videos
train_folder = 'train_75'
val_folder = 'val_25'
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Copy train and val videos to new folders
for video in truth_train:
    shutil.copy(os.path.join('/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/train_videos', video), os.path.join(train_folder, video))

for video in truth_val:
    shutil.copy(os.path.join('/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/train_videos', video), os.path.join(val_folder, video))

for video in lie_train:
    shutil.copy(os.path.join('/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/train_videos', video), os.path.join(train_folder, video))

for video in lie_val:
    shutil.copy(os.path.join('/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/train_videos', video), os.path.join(val_folder, video))


In [94]:
headers = ['VideoID']
X_train = pd.DataFrame(X_train, columns=headers)
X_train

,VideoID
0,BM017_1PT
1,WM027_2NL
2,WF024_3NT
3,WM026_1PT
4,WF024_1PT
...,...
251,WF011_1PT
252,BF027_4PL
253,BM009_3NT
254,WM011_3NT


In [95]:
X_train['VideoID'] = X_train['VideoID'] + '.wmv'
X_train


,VideoID
0,BM017_1PT.wmv
1,WM027_2NL.wmv
2,WF024_3NT.wmv
3,WM026_1PT.wmv
4,WF024_1PT.wmv
...,...
251,WF011_1PT.wmv
252,BF027_4PL.wmv
253,BM009_3NT.wmv
254,WM011_3NT.wmv


In [30]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Specify the folder containing all the videos
folder_path = '/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/Train_val_test Split/test_videos'

# List all video files in the folder
video_files = [f for f in os.listdir(folder_path) if f.endswith('.wmv')]

# Initialize empty numpy array to store coordinates
num_videos = len(video_files)
all_coordinates = np.zeros((num_videos, 20, 468, 2))  # Assuming 468 landmarks and 2 coordinates (X, Y)

# Process each video
for i, video_file in enumerate(video_files):
    video_path = os.path.join(folder_path, video_file)
    # Initialize OpenCV VideoCapture for current video
    cap = cv2.VideoCapture(video_path)
    
    # Get total number of frames and calculate interval to extract frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval = total_frames // 20
    
    # Process frames for current video
    frame_count = 0
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % interval == 0:
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Detect face landmarks
            results = face_mesh.process(frame_rgb)
            if results.multi_face_landmarks:
                # Extract landmarks
                landmarks = results.multi_face_landmarks[0].landmark
                for j, landmark in enumerate(landmarks):
                    all_coordinates[i, frame_index, j] = [landmark.x, landmark.y]
                 
                frame_index += 1
                if frame_index == 20:
                    break
    
    # Release video capture for current video
    cap.release()

# Save the numpy array to a file
np.save('test_facemesh_coordinates.npy', all_coordinates)


I0000 00:00:1714552816.949485   67704 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1714552816.951177   79498 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.2-manjaro1.1), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


In [138]:
import numpy as np

# Load the .npy file into memory
test_var = np.load('test_facemesh_with_veracity.npy')

# Print the array

print(test_var)


[[[[0.37571236 0.64782476 1.        ]
   [0.37919608 0.61058551 1.        ]
   [0.37672618 0.61598444 1.        ]
   ...
   [0.4002409  0.48942256 1.        ]
   [0.46762076 0.46741202 1.        ]
   [0.47430792 0.4528625  1.        ]]

  [[0.37777516 0.58643359 1.        ]
   [0.38420504 0.54290432 1.        ]
   [0.38044479 0.55393082 1.        ]
   ...
   [0.40572381 0.43033728 1.        ]
   [0.47384003 0.41244158 1.        ]
   [0.48168448 0.39972213 1.        ]]

  [[0.42043906 0.54319102 1.        ]
   [0.43307319 0.49952865 1.        ]
   [0.42521986 0.51195711 1.        ]
   ...
   [0.44710743 0.39020121 1.        ]
   [0.49489906 0.3741211  1.        ]
   [0.5015856  0.36421561 1.        ]]

  ...

  [[0.49883777 0.41441336 1.        ]
   [0.50053048 0.37155139 1.        ]
   [0.49943697 0.38534674 1.        ]
   ...
   [0.51748919 0.29588535 1.        ]
   [0.57233906 0.28536299 1.        ]
   [0.57825494 0.27568579 1.        ]]

  [[0.         0.         0.        ]
   [0. 

In [1]:
# Filter the array based on veracity
veracity_0_indices = np.where(test_var[:, :, :, 2] == 0)

# Extract coordinates where veracity is 0
coordinates_veracity_0 = test_var[veracity_0_indices]

# Print the coordinates
print(coordinates_veracity_0)


NameError: name 'np' is not defined

In [25]:
##### CREATE TRAIN AND TEST CSV


import os
import csv

# Specify the folder containing the videos
folder_path = 'val_25'

# List all video files in the folder and sort them
video_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.wmv')])

# Write the video names to a CSV file
csv_file = 'val_25.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['VideoID'])  # Write header
    for video in video_files:
        writer.writerow([video])

print(f"Video names have been written to {csv_file}.")


Video names have been written to val_25.csv.


In [26]:
##### REMOVE SUFFIX .WMV

import csv
import os

# Remove suffix from VideoID in the CSV file
def remove_suffix(csv_file):
    updated_rows = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            row['VideoID'] = row['VideoID'].rstrip('.wmv')
            updated_rows.append(row)
    return updated_rows

# Write the updated rows to the same CSV file
def overwrite_csv(updated_rows, csv_file):
    fieldnames = updated_rows[0].keys()
    with open(csv_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(updated_rows)

# Example usage
input_csv = 'val_25.csv'

updated_rows = remove_suffix(input_csv)
overwrite_csv(updated_rows, input_csv)

print(f"CSV file {input_csv} has been updated to remove the '.wmv' suffix.")


CSV file val_25.csv has been updated to remove the '.wmv' suffix.


In [27]:
##### ADD VALUES OF VERACITY

import pandas as pd

# Load train_videos CSV file with video names
train_videos_file = 'val_25.csv'
train_videos_df = pd.read_csv(train_videos_file)

# Load CSV file containing all veracities
veracities_file = '/home/mayur/Desktop/Machine Learning/ML Project (Lie Detection)/Facemesh/Model Prediction/VideoList_Veracity.csv'  # Update with your file name
veracities_df = pd.read_csv(veracities_file)

# Merge train_videos_df with veracities_df based on video names
merged_df = pd.merge(train_videos_df, veracities_df, on='VideoID', how='left')

# Overwrite the train_videos CSV file with veracities added
merged_df.to_csv(train_videos_file, index=False)

print(f"Veracities have been added to {train_videos_file} and saved.")


Veracities have been added to val_25.csv and saved.
